## Semester 1 Project Submission

Please fill out:
* Student name: 
* Instructor name: 


In [4]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
import pandas as pd
import sqlite3
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline
# You'll need to unzip the db locally.
conn = sqlite3.connect('zippedData/im.db')

# Habib Khadri

# Nick Kirbabas

In [36]:
df_budget = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [12]:
df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t')

In [21]:
df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [19]:
df.isna().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

This file has many null values with some columns almost completely unusable


 Looking at the SQL db



Narrowing down list of directors to ones that have over 10 movies (want experienced directors) and have films in the US, our largest target audience

In [132]:
pd.read_sql('''
SELECT distinct primary_name
FROM principals
JOIN persons USING (person_id)
JOIN movie_akas USING (movie_id)
WHERE category == 'director' AND region == 'US'
GROUP BY primary_name
HAVING count(person_id) > 10
ORDER BY count(person_id) DESC
''',conn)

,primary_name
0,Ansel Faraj
1,Larry Rosen
2,Charlie Minn
3,Jon M. Chu
4,Dustin Ferguson
...,...
94,Deon Taylor
95,Dennis Dugan
96,Dennis Devine
97,Darren Lynn Bousman


Adding previous query as subquery to find the average rating of the directors' movies. This should help us in our recommendation of which directors to choose for the film

In [17]:
pd.read_sql('''
Select persons.primary_name as Director, avg(averagerating) as ratings
from principals 
JOIN persons ON persons.person_id = principals.person_id
JOIN movie_basics as mb ON mb.movie_id = principals.movie_id
JOIN movie_ratings as mr ON mr.movie_id = mb.movie_id
WHERE category == 'director' AND persons.death_year is NULL AND primary_name IN 
(
SELECT distinct primary_name
FROM principals
JOIN persons USING (person_id)
JOIN movie_akas USING (movie_id)
WHERE category == 'director' AND region == 'US'
GROUP BY primary_name
HAVING count(person_id) > 10
ORDER BY count(person_id) DESC
)
GROUP BY Director
HAVING sum(numvotes) > 5000
ORDER BY ratings DESC
LIMIT 15
''',conn )

,Director,ratings
0,Christopher Nolan,8.425000
1,Anthony Russo,8.225000
2,Joe Russo,7.820000
3,Martin Scorsese,7.657143
4,J.J. Abrams,7.566667
5,Steven Spielberg,7.228571
6,Alex Gibney,7.173333
7,Justin Lin,7.166667
8,Joseph Kosinski,7.166667
9,James Wan,7.116667


(NOTE: adding a conditional to the sum of votes on each movie prevents documentaries, which have high ratings but low votes, from skewing results)

Similar query as previous but this time looking at Actors

In [15]:
pd.read_sql('''
Select persons.primary_name as Actor, avg(averagerating) as ratings
from principals 
JOIN persons ON persons.person_id = principals.person_id
JOIN movie_basics as mb ON mb.movie_id = principals.movie_id
JOIN movie_ratings as mr ON mr.movie_id = mb.movie_id
WHERE category == 'actor' AND persons.death_year is NULL AND primary_name IN 
(
SELECT distinct primary_name
FROM principals
JOIN persons USING (person_id)
JOIN movie_akas USING (movie_id)
WHERE category == 'actor' AND region == 'US'
GROUP BY primary_name
HAVING count(person_id) > 10
ORDER BY count(person_id) DESC
)
GROUP BY Actor
HAVING sum(numvotes) > 1000
ORDER BY ratings DESC
LIMIT 15
''',conn )

,Actor,ratings
0,Leonardo DiCaprio,8.090000
1,Robert Downey Jr.,7.530769
2,Andrew Garfield,7.472727
3,Ian McKellen,7.412500
4,Tom Hardy,7.361538
5,Mark Ruffalo,7.323529
6,Brad Pitt,7.208333
7,Hugh Jackman,7.207692
8,Jeremy Renner,7.181818
9,Richard Armitage,7.172727


And now actresses 

In [20]:
pd.read_sql('''
Select persons.primary_name as Actress, avg(averagerating) as ratings
from principals 
JOIN persons ON persons.person_id = principals.person_id
JOIN movie_basics as mb ON mb.movie_id = principals.movie_id
JOIN movie_ratings as mr ON mr.movie_id = mb.movie_id
WHERE category == 'actress' AND persons.death_year is NULL AND primary_name IN 
(
SELECT distinct primary_name
FROM principals
JOIN persons USING (person_id)
JOIN movie_akas USING (movie_id)
WHERE category == 'actress' AND region == 'US'
GROUP BY primary_name
HAVING count(person_id) > 10
ORDER BY count(person_id) DESC
)
GROUP BY Actress
HAVING sum(numvotes) > 1000
ORDER BY ratings DESC
LIMIT 15
''',conn )

,Actress,ratings
0,Amy Adams,7.128571
1,Bryce Dallas Howard,7.075000
2,Helena Bonham Carter,7.018182
3,Scarlett Johansson,6.992308
4,Rachel McAdams,6.933333
5,Rooney Mara,6.929412
6,Zoe Saldana,6.925000
7,Margot Robbie,6.915385
8,Jennifer Lawrence,6.811111
9,Cate Blanchett,6.800000


# Seerat Mutneja

# Sidhya Raman

# Douglas Riggs

In [39]:
pd.read_sql('''
SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';
''',conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


# Andrew Yang